# Distance analysis for CD8T cells

In [ ]:
import os
import pandas as pd
import numpy as np
import skimage.io as io
import xarray as xr

In [ ]:
cell_tab_path = "../data/tables/cell_table_size_normalized.csv"
nimbus_tab_path = "../data/tables/nimbus_output.csv"
dist_mat_dir = "../data/spatial_analysis/dist_mats"
seg_dir = "../data/segmentation_masks"

## Count number of cell types

In [ ]:
cell_tab = pd.read_csv(cell_tab_path)
all_fovs = np.unique(cell_tab['fov'])

all_windows = [(0,10), (10,20), (20,30), (30,40), (40,50), (50,60), (60,70), (70,80), (80,90), (90,100)]
all_window_counts = pd.DataFrame(columns=['fov','index_cell_label','cell_meta_cluster','count','window'])

for fov in all_fovs:
    print(fov)
    one_fov_tab = cell_tab[(cell_tab['fov']==fov) & (cell_tab['cell_meta_cluster']=='CD8T')]
    one_fov_labs = one_fov_tab['label'].values
    one_fov_dist = xr.load_dataarray(os.path.join(dist_mat_dir, str(fov) + '_dist_mat.xr'))
    one_fov_dist = one_fov_dist.loc[one_fov_dist.dim_0.isin(one_fov_labs),:]
    
    one_fov_all = cell_tab[cell_tab['fov']==fov][['label','cell_meta_cluster']]

    for lower_um,upper_um in all_windows:

        lower_px = lower_um * (2048/800)
        upper_px = upper_um * (2048/800)

        mask = (one_fov_dist > lower_px) & (one_fov_dist <= upper_px)
        xr_indices = np.where(mask.values)
        mask_dat = pd.DataFrame({
            'index_cell_label': one_fov_dist.dim_0.values[xr_indices[0]],
            'label': one_fov_dist.dim_1.values[xr_indices[1]]
        })

        mask_dat = pd.merge(mask_dat, one_fov_all, on='label')
        mask_counts = mask_dat.groupby(['index_cell_label','cell_meta_cluster']).size().reset_index(name='count')
        mask_counts['window'] = upper_um
        mask_counts['fov'] = fov

        all_window_counts = pd.concat([all_window_counts, mask_counts])
        
all_window_counts.to_csv("../data/cd8t_dist_counts.csv", index=False)

## Get average expression

In [ ]:
cell_tab = pd.read_csv(cell_tab_path)
all_fovs = np.unique(cell_tab['fov'])

all_windows = [(0,10), (10,20), (20,30), (30,40), (40,50), (50,60), (60,70), (70,80), (80,90), (90,100)]

out_dir = "../data/spatial_analysis/cd8t_dist_exp_dfs"
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

markers = ['CD11c', 'CD14', 'CD163', 'CD20', 'CD21', 'CD31', 'CD3e', 'CD4', 'CD45', 'CD45RO',
           'CD56', 'CD68', 'CD69', 'CD86', 'CD8a', 'CXCR5', 'Calprotectin', 'Caspase1', 'FOXP3', 'Galectin9',
           'Glut1', 'GranzymeB', 'HH3', 'HLA1classABC', 'HLADR', 'ICOS', 'IDO', 'IFNg', 'Ki67', 'Lag3',
           'MastCellTryptase', 'NLRP3', 'PD1', 'PDL1', 'SMA', 'TCF1TCF7', 'TIGIT', 'TIM3', 'Vimentin']

done_fovs = os.listdir(out_dir)
done_fovs = [x.replace(".csv","") for x in done_fovs]
keep_fovs = [x for x in all_fovs if x not in done_fovs]

for fov in keep_fovs:
    print(fov)
    one_fov_df = pd.DataFrame(columns=['fov','index_cell_label','mean_exp','window','marker'])
    
    one_fov_tab = cell_tab[(cell_tab['fov']==fov) & (cell_tab['cell_meta_cluster']=='CD8T')]
    one_fov_labs = one_fov_tab['label'].values
    one_fov_dist = xr.load_dataarray(os.path.join(dist_mat_dir, str(fov) + '_dist_mat.xr'))
    one_fov_dist = one_fov_dist.loc[one_fov_dist.dim_0.isin(one_fov_labs),:]
    
    one_fov_all = cell_tab[cell_tab['fov']==fov][['label']+markers]

    for lower_um,upper_um in all_windows:
        lower_px = lower_um * (2048/800)
        upper_px = upper_um * (2048/800)

        mask = (one_fov_dist > lower_px) & (one_fov_dist <= upper_px)
        xr_indices = np.where(mask.values)
        mask_dat = pd.DataFrame({
            'index_cell_label': one_fov_dist.dim_0.values[xr_indices[0]],
            'label': one_fov_dist.dim_1.values[xr_indices[1]]
        })

        mask_dat = pd.merge(mask_dat, one_fov_all, on='label')
        mask_dat_melt = pd.melt(mask_dat, id_vars=['index_cell_label','label'], value_vars=markers, var_name='marker', value_name='exp_value')
        mask_exp = mask_dat_melt.groupby(['index_cell_label','marker'])['exp_value'].mean().reset_index(name='mean_exp')
        mask_exp['window'] = upper_um
        mask_exp['fov'] = fov
        one_fov_df = pd.concat([one_fov_df, mask_exp])
    
    one_fov_df.to_csv(os.path.join(out_dir, fov+".csv"), index=False)